# Initialization

In [1]:
%load_ext autoreload
%autoreload 2
from context import rmlib
import rmlib.rmtools as rm
from config import robotConfig

robot = rmlib.robot.Robot( robotConfig )

#Imports 
from time import sleep
import numpy as np
import rmlib
from rmlib.rmtools.assembly_trees.asm_tree_Basic import augment_RM

augment_RM( robot )

%store -r

####### LIBS #######################################################################################################################################

# Import Pytrees
import py_trees
from py_trees.tests import Timeout_Success
from py_trees.composites import Sequence , Selector
from py_trees.decorators import FailureIsSuccess , SuccessIsFailure

# import 
from rmlib.rmtools.assembly_trees.asm_tree_Basic import *

from rmlib.rmtools.assembly_trees.asm_tree_FT_based import ( 
    Bias_Wrist , Spiral_Insert , Tilt_Insert , Spin_Press , Move_to_Contact , SpiralStep
)
    
from rmlib.rmtools.assembly_trees.asm_tree_drill import *

from rmlib.rmtools.assembly_trees.asm_tree_logic_flow import Run_to_X_Failures_DECO

import math
from pmath import *

from utils import is_matx_list
from math import radians

print( "Libs loaded!" )

from task_parameters import taskParams , _TB_side , _M8_TB_height , _M6_TB_height , _M4_TB_height

_DRILL_CONTACT_PRESS = 11.0

import pickle , traceback

def save_names_to_PKL( pklPath , nameStringList ):
    pklStruct = {}
    print( vars() )
    for name in nameStringList:
        pklStruct[ name ] = globals()[ name ]
        
    with open( pklPath , "wb" ) as f:
        try:
            print( "Pickling" , pklPath , end = '... ' )
            pickle.dump( pklStruct , f )
            print( "Success!" )
        except Exception as er:
            print( "FAILURE! :" , er )
            traceback.print_exc()
    print( "### Completed Writing" , pklPath , "###" )
    
def load_names_from_PKL_to_global( pklPath , nameStringList ):
    n = 0
    with open( pklPath , "rb" ) as f:
        try:
            print( "Unpickling" , pklPath , end = '... ' )
            pklStruct = pickle.load( f )
            print( "Got: " , type( pklStruct ) , "with" , len( pklStruct ) , "elements" )
            globDict = globals()
            for name in nameStringList:
                try:
                    globals()[ name ] = pklStruct[ name ]
                    n += 1
                except:
                    print( "Could not load '" + str( name ) + "'! Continue ..." )
        except Exception as er:
            print( "FAILURE! :" , er )
            traceback.print_exc()
            
    print( "### Read" , n , "vars from" , pklPath , "into global dict ###" )

load_names_from_PKL_to_global( 
    pklPath = "taughtPoses.pkl" , 
    nameStringList = [  
        'M8_GraspPose' , 'M6_GraspPose' , 'M4_GraspPose' , 'GearShaft1_Setdown' , 'GearShaft2_Setdown' , 
        'BNC_Ma_Setdown' , 'KeySm_Setdown' , 'KeyLg_Setdown' , 'ShaftSm_Setdown' , 'ShaftLg_Setdown' , 
        'KeyLg_Grasp' , 'ShaftLg_Grasp' , 'BNC_Fe_Grasp' , 'ShaftSm_Grasp' , 
        'GearLg_Grasp' , 'GearLg_Grasp' , 'Ether_setdown' , 
        'M8_1_mag' , 'M8_2_mag' , 'M8_3_mag' , 'M8_4_mag' , 'M8_5_mag' , 'M8_6_mag' ,
        'M6_1_mag' , 'M6_2_mag' , 'M6_3_mag' , 'M6_4_mag' , 'M6_5_mag' , 'M6_6_mag' ,
        'M4_1_mag' , 'M4_2_mag' , 'M4_3_mag' , 'M4_4_mag' , 'M4_5_mag' , 'M4_6_mag' ,
        'releasePose' , 'SAFE_POSE' , 'midDrillPose' ,
        '_LRG_GEAR_DEX' , '_SML_GEAR_DEX', 'aboveTableDrill'
    ]
)

load_names_from_PKL_to_global( 
    pklPath = "dct.pkl" , 
    nameStringList = [ "dct" ]
)


## BIAS TEST ##
if 0:
    for i in range(40):
        print('##', i+1, '##')
        before = robot.ft.get_wrist_force()
        bgn    = time.time()
        robot.ft.bias_wrist_force()
        print( time.time() - bgn , "[s] bias time" )
        after  = robot.ft.get_wrist_force()
        print( after )
        print( "Diff:" , np.linalg.norm( np.subtract( after, before ) ) , '\n\n' )

Loaded: /home/nvidia/dev_rmstudio/rmlib/rmlib/rmtools Exists?: True
Loaded: /home/nvidia/dev_rmstudio/rmlib Exists?: True
Loading {'active_components': {'ft': ['ft_config'], 'hand': ['hand_config'], 'arm': ['arm_config'], 'camera': ['cam_config']}, 'my_components': {'hand_config': {'module_name': 'hands.smarthand', 'class_name': 'SmartHand', 'finger_depth': 0.014, 'finger_width_outer': 0.015, 'finger_length': 0.0415, 'finger_width_inner': 0.0}, 'arm_config': {'max_joint_speed': 1.05, 'class_name': 'UR5', 'ip_address': '192.168.0.6', 'default_linear_accel': 0.8, 'max_linear_speed': 0.25, 'module_name': 'arms.ur5', 'default_joint_speed': 0.7, 'default_linear_speed': 0.1, 'max_linear_accel': 1.2, 'max_joint_accel': 1.4, 'xmlrpc_port': '8003', 'default_joint_accel': 0.8}, 'ft_config': {'module_name': 'sensors.ftsensor_optoforce', 'class_name': 'OptoForce', 'ip_address': '192.168.0.3', 'filter': {'alpha': 0.012, 'beta': 0.035}, 'dataHz': 100.0}, 'cam_config': {'pc_cam_offset': [-0.037, -0.0

# Setup poses

In [ ]:
# record coordinates of the E
SAFE_POSE=robot.arm.get_tcp_pose()

In [2]:
import pickle
# load coordinates of the E
with open('coords.pkl','rb') as f:  # Python 3: open(..., 'rb')
    E1, E2, E3, E4, E5, E6, SAFE_POSE, midDrillPose = pickle.load(f)

In [ ]:
E2 = translate_pose( 
    E1 , 
    [ 0.0 , 0, -0.1 ] , # Translation vector
    dir_pose = 'origin' # { 'origin': lab frame, 'self': relative the argument pose }
)

In [46]:
# write coordinates of the E
with open('coords.pkl', 'wb') as f: 
    pickle.dump([E1, E2, E3, E4, E5, E6, SAFE_POSE, midDrillPose], f)

In [38]:
moveNode = Move_Arm( pose  = E2 , # 4x4 homogeneous coord pose
                     mode  = 'j' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.125 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB

run_BT_until_done( moveNode )

### RUN BT Move_Arm ###

Root node Move_Arm succeeded!

--> Move_Arm [o]


Run completed! with status Status.SUCCESS after 21.40099859237671 seconds.



### BT SESSION OVER ###


# DEBUG Bad 'E' Poses

In [ ]:
import pickle
# load coordinates of the E
with open('coords.pkl','rb') as f:  # Python 3: open(..., 'rb')
    E1, E2, E3, E4, E5, E6, SAFE_POSE, midDrillPose = pickle.load(f)

In [ ]:
%matplotlib inline
from context import rmlib
from Plotting import *
from pmath import *
f = 1.65
fig,ax = fig_3d( [12*f,9*f] )

plot_pose_axes_mpl( ax , origin_pose() , 0.160 , labelStr = 'O' )

V = []

p_alt = 1

for i, pose in enumerate( [E1, E2, E3, E4, E5, E6] ):
    print( pose )
    plot_pose_axes_mpl( ax , pose , 0.060 , labelStr = 'E' + str(i+1) )
    print( pose_mtrx_to_vec( pose, alt = p_alt ) )
    if 0:
        plot_pose_axes_mpl( ax , 
                            pose_vec_to_mtrx( pose_mtrx_to_vec( pose, alt = p_alt ) ), 
                            0.080 , labelStr = 'V' + str(i+1) )
    print()
    
axes_equal( ax )
show_3d()

In [ ]:
np.set_printoptions(precision=3,suppress=True)
robot.arm.get_tcp_pose()

In [ ]:
E1

# Define BTs

## Drilling BT

In [3]:
## This tree drills a hole in Y-direction by gently tapping
tapTree = Sequence( memory = 1 ) 

# 1. Approach the wall at a **gentle speed** and stop when a small reaction force is observed 
tapTree.add_child(
    Move_to_Contact( 
        Fmag      = 5.0 , # Reaction force that triggers stop, Forces below 1.5N aren't reliably detected
        relMove   = [0.0, 0.2, 0.0] , # Specify a move the goes *beyond* where you expect to end up
        biasWrist = 1 , # Bias wrist before beginning motion (Sorry, currently this takes 1-8 seconds)
        mode      = 'l' , 
        speed     = 0.040 , accel = 0.04 , # **gentle speed**
        pull      = 0 , # For pulling actions, reaction force tugs on hand rather than pushes
        ctrl      = robot
    ) # NOTE: This behav. may be used for absolute pose if an argument for `pose` is provided
)

# 2. Back up
tapTree.add_child(
    Move_Arm_Relative( 
        translation = [ 0.0, -0.005, 0.0 ] , # Trans. Vec
        rotation    = [0.0,0.0,0.0] , # Euler angles
        ctrl = robot
    )
)

# 3. set_drill_torque(200)
tapTree.add_child(  Set_Drill_Torque( drillTorque = 200 , ctrl = robot )  )

# 4. Activate Drill
tapTree.add_child(
    Turn_Drill_CW( ctrl = robot )
)

# 5. Maintain Pressure for Dwell Time (Assumes vertical pressure)
tapTree.add_child(
    Maintain_Z_Pressure( 
        timeOut_s = 10.0 , # Time to dwell [s]
        plungeMove = 1 , # Flag for move-to-contact
        plungeDepth=0.030 , # Distance to move down
        descendSpeed=0.030 , 
        zPress=20.0 , # [N] Pressure to maintain, ()
        maxZStep=0.010 , biasWrist=1 ,
        ctrl = robot
    )
)

# 6. Reverse drill to avoid embarrassing loss of bit
tapTree.add_child(
    Turn_Drill_CCW( ctrl = robot )
)

# 7. Back up
tapTree.add_child(
    Move_Arm_Relative( 
        translation = [ 0.0, -0.1, 0.0 ] , # Trans. Vec
        rotation    = [0.0,0.0,0.0] , # Euler angles
        speed     = 0.060 , accel = 0.06 , # **gentle speed**
        ctrl = robot
    )
)

# 8. Stop drill
tapTree.add_child(
    Halt_Drill( ctrl = robot )
)

UUID('1be717f8-e830-4737-ac87-bd8aa8399c83')

In [4]:
## This tree inserts a hose by gently tapping
insertTree = Sequence( memory = 1 ) 

# 1. Approach the wall at a **gentle speed** and stop when a small reaction force is observed 
insertTree.add_child(
    Move_to_Contact( 
        Fmag      = 5.0 , # Reaction force that triggers stop, Forces below 1.5N aren't reliably detected
        relMove   = [0.0, 0.2, 0.0] , # Specify a move the goes *beyond* where you expect to end up
        biasWrist = 1 , # Bias wrist before beginning motion (Sorry, currently this takes 1-8 seconds)
        mode      = 'l' , 
        speed     = 0.040 , accel = 0.04 , # **gentle speed**
        pull      = 0 , # For pulling actions, reaction force tugs on hand rather than pushes
        ctrl      = robot
    ) # NOTE: This behav. may be used for absolute pose if an argument for `pose` is provided
)

# 5. Maintain Pressure for Dwell Time (Assumes vertical pressure)
insertTree.add_child(
    Maintain_Z_Pressure( 
        timeOut_s = 10.0 , # Time to dwell [s]
        plungeMove = 1 , # Flag for move-to-contact
        plungeDepth=0.030 , # Distance to move down
        descendSpeed=0.030 , 
        zPress=5.0 , # [N] Pressure to maintain, ()
        maxZStep=0.010 , biasWrist=1 ,
        ctrl = robot
    )
)

# 7. Back up
insertTree.add_child(
    Move_Arm_Relative( 
        translation = [ 0.0, -0.1, 0.0 ] , # Trans. Vec
        rotation    = [0.0,0.0,0.0] , # Euler angles
        speed     = 0.060 , accel = 0.06 , # **gentle speed**
        ctrl = robot
    )
)



UUID('ad7918fc-6a81-475b-9229-9abf41e1a9ee')

## Main Drilling BT

In [10]:
# Adjust this up in increments of 0.5-1.0 if you are having trouble picking up drill
_DRILL_CONTACT_PRESS = 11.0 # Too aggressive and holders move out of place over time

_LONG_DRILL_SAFE_Z = 0.250 # <-- VERY IMPORTANT - AVOID BREAKING DRILL

testTree =  Sequence( memory = 1 ) 

testTree.add_child(
    Unholster_Drill( 
    M4_GraspPose , 
    'M4' , 
    betweenPose = midDrillPose , 
    zApproach=0.20, zClose=0.040, zRetry=0.030 , Tpliant = 0.3 , 
    wdthNarrow = 0.038 , maxIter=5, drillPress = _DRILL_CONTACT_PRESS , 
    zFree=_LONG_DRILL_SAFE_Z , # <-- VERY IMPORTANT - AVOID BREAKING DRILL
    ctrl = robot 
    )
)

testTree.add_child(
    Move_Arm( pose  = SAFE_POSE , # 4x4 homogeneous coord pose
                     mode  = 'j' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.5 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB
)

for HoleLocation in [E1,E2,E3,E4,E5,E6]:

    testTree.add_child(
        Move_Arm( pose  = HoleLocation , # 4x4 homogeneous coord pose
                         mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                         speed = 0.5 , 
                         accel = 0.35 , 
                         ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB
    )

    testTree.add_child(tapTree)

testTree.add_child(
    Move_Arm( pose  = SAFE_POSE , # 4x4 homogeneous coord pose
                     mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.5 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB
)


testTree.add_child(
    Holster_Drill( 
        M4_GraspPose , 
        'M4' , 
        betweenPose = midDrillPose , 
        zApproach=0.20, 
        zClose=0.040, 
        Tpliant = 0.3 ,  
        wdthNarrow = 0.038 , maxIter=5, drillPress=_DRILL_CONTACT_PRESS , 
        zFree= _LONG_DRILL_SAFE_Z , # <-- VERY IMPORTANT - AVOID BREAKING DRILL
        driverLen = 0.160 ,
        ctrl = robot 
    )
)

run_BT_until_done( testTree , N = 90000 , 
                   tickPause = 0.25 , 
                   breakOnFailure = 1 , breakOnSuccess = 1 , 
                   Nverb = 100 )

### RUN BT Sequence ###

M4 checked OUT!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 14.480835914611816 seconds!
Halt_Drill: Ended with drill ATTACHED!
[-] Sequence [*]
    [-] Unholster_Drill [o]
        --> Move_Arm [o]
        --> Move_Arm [o]
        --> Set_Grip_Torque [o]
        --> Set_Fingers [o]
        --> Move_Arm [o]
        --> Set_Fingers [o]
        --> Move_to_Contact [o]
        --> Set_Grip_Torque [o]
        [o] Recover_or_Quit [o]
            -^- Run_to_X_Failures_DECO [o]
                [o] Attached_Recover [o]
                    --> COND_Drill_Attached [o]
                    [-] Recovery_Sequence [-]
                        --> Set_Fingers [-]
                        --> Set_Grip_Torque [-]
                        --> Move_Arm [-]
                        --> Set_Fingers [-]
                        --> Move_to_Contact [-]
                        --> Set_Grip_Torque [-]
     

In [5]:
# Adjust this up in increments of 0.5-1.0 if you are having trouble picking up drill
_DRILL_CONTACT_PRESS = 11.0 # Too aggressive and holders move out of place over time

_LONG_DRILL_SAFE_Z = 0.250 # <-- VERY IMPORTANT - AVOID BREAKING DRILL

testTree =  Sequence( memory = 1 ) 

testTree.add_child(
    Unholster_Drill( 
    M6_GraspPose , 
    'M6' , 
    betweenPose = midDrillPose , 
    zApproach=0.20, zClose=0.040, zRetry=0.030 , Tpliant = 0.3 , 
    wdthNarrow = 0.038 , maxIter=5, drillPress = _DRILL_CONTACT_PRESS , 
    zFree=_LONG_DRILL_SAFE_Z , # <-- VERY IMPORTANT - AVOID BREAKING DRILL
    ctrl = robot 
    )
)

testTree.add_child(
    Move_Arm( pose  = SAFE_POSE , # 4x4 homogeneous coord pose
                     mode  = 'j' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.5 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB
)

for HoleLocation in [E1,E2,E3,E4,E5,E6]:

    testTree.add_child(
        Move_Arm( pose  = HoleLocation , # 4x4 homogeneous coord pose
                         mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                         speed = 0.5 , 
                         accel = 0.35 , 
                         ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB
    )

    testTree.add_child(insertTree)

testTree.add_child(
    Move_Arm( pose  = SAFE_POSE , # 4x4 homogeneous coord pose
                     mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.5 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB
)


testTree.add_child(
    Holster_Drill( 
        M6_GraspPose , 
        'M6' , 
        betweenPose = midDrillPose , 
        zApproach=0.20, 
        zClose=0.040, 
        Tpliant = 0.3 ,  
        wdthNarrow = 0.038 , maxIter=5, drillPress=_DRILL_CONTACT_PRESS , 
        zFree= _LONG_DRILL_SAFE_Z , # <-- VERY IMPORTANT - AVOID BREAKING DRILL
        driverLen = 0.160 ,
        ctrl = robot 
    )
)

run_BT_until_done( testTree , N = 90000 , 
                   tickPause = 0.25 , 
                   breakOnFailure = 1 , breakOnSuccess = 1 , 
                   Nverb = 100 )

### RUN BT Sequence ###

M6 checked OUT!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 13.782416343688965 seconds!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 14.942351818084717 seconds!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 13.767829656600952 seconds!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 13.911924123764038 seconds!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 13.760231256484985 seconds!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 13.838942289352417 seconds!

In [8]:
   

run_BT_until_done( Holster_Drill( 
        M4_GraspPose , 
        'M4' , 
        betweenPose = midDrillPose , 
        zApproach=0.20, 
        zClose=0.040, 
        Tpliant = 0.3 ,  
        wdthNarrow = 0.038 , maxIter=5, drillPress=_DRILL_CONTACT_PRESS , 
        zFree= _LONG_DRILL_SAFE_Z , # <-- VERY IMPORTANT - AVOID BREAKING DRILL
        driverLen = 0.160 ,
        ctrl = robot             
    ) , N = 90000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 , Nverb = 100 )

### RUN BT Holster_Drill ###

[-] Holster_Drill [x]
    [-] COND_Holding_Drill [x]
        --> COND_Drill_Checked_Out [x]
        --> COND_Drill_Attached [-]
    --> COND_Drill_Attached [-]
    [-] Jog_Safe [-]
        --> Move_Arm [-]
        --> Move_Arm [-]
    --> Move_Arm [-]
    --> Move_Arm [-]
    --> Move_Arm [-]
    --> Move_to_Contact [-]
    --> Set_Grip_Torque [-]
    --> Set_Fingers [-]
    --> Move_Arm [-]
    --> Set_Fingers [-]
    --> Move_Arm [-]
    -^- Negator_DECO [-]
        --> COND_Drill_Attached [-]
    [-] Jog_Safe [-]
        --> Move_Arm [-]
        --> Move_Arm [-]
    --> Set_Grip_Torque [-]
    --> Check_OUT_Drill [-]


Run completed! with status Status.FAILURE after 9.822951555252075 seconds.



### BT SESSION OVER ###


In [ ]:
E1

# Point Clouds

In [3]:
from cloud_proc.cloud_pipeline import Cloud_Pipeline
CPL = Cloud_Pipeline()

Cloud_Pipeline object created from components:  [<class 'cloud_proc.cloud_capturing.Cloud_Capturing'>, <class 'cloud_proc.cloud_poses.Cloud_Poses'>, <class 'cloud_proc.cloud_segmentation.Cloud_Segmentation'>, <class 'cloud_proc.cloud_processing.Cloud_Processing'>, <class 'cloud_proc.viewer_threejs.Viewer'>]


In [9]:
moveNode = Move_Arm( pose  = E5 , # 4x4 homogeneous coord pose
                     mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.125 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB

run_BT_until_done( moveNode )

### RUN BT Move_Arm ###

Root node Move_Arm succeeded!

--> Move_Arm [o]


Run completed! with status Status.SUCCESS after 0.23032283782958984 seconds.



### BT SESSION OVER ###


In [21]:
robot.hand.set_finger_width(0.12)

True

In [30]:
robot.camera.set_disparity_shift_dist( 0.35) # Fetch Z
# Capture pnt Cloud and transform to lab frame
cloudInLabFrame = transform_points( robot.camera.get_cloud() , robot.get_base_to_camera_pose() )

set_disparity_shift_dist: Got distance: 0.35 , Apply Offset: -0.079


In [31]:
view = CPL.PC_Viewer()
view.add_cloud( cloudInLabFrame )
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-2.0535437967342993, 4.320151336484518, 3.35…

In [45]:
E2[2,3]=E5[2,3]

In [40]:
E5

array([[ 0.99849694,  0.0472845 ,  0.02771333,  0.05545081],
       [-0.03031048,  0.05512346,  0.99801938,  0.72335381],
       [ 0.04566319, -0.9973593 ,  0.05647382,  0.32991594],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [6]:
# Adjust this up in increments of 0.5-1.0 if you are having trouble picking up drill
_DRILL_CONTACT_PRESS = 11.0 # Too aggressive and holders move out of place over time

_LONG_DRILL_SAFE_Z = 0.250 # <-- VERY IMPORTANT - AVOID BREAKING DRILL

testTree =  Sequence( memory = 1 ) 


for HoleLocation in [E2,E3,E4,E5,E6]:

    testTree.add_child(
        Move_Arm( pose  = HoleLocation , # 4x4 homogeneous coord pose
                         mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                         speed = 0.5 , 
                         accel = 0.35 , 
                         ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB
    )

    testTree.add_child(tapTree)

testTree.add_child(
    Move_Arm( pose  = SAFE_POSE , # 4x4 homogeneous coord pose
                     mode  = 'l' ,  # { 'l': linear in task space, 'j': linear in joint space }
                     speed = 0.5 , 
                     accel = 0.35 , 
                     ctrl  = robot ) # Ref to `rmlib` object, stored in `robot` in this NB
)


testTree.add_child(
    Holster_Drill( 
        M4_GraspPose , 
        'M4' , 
        betweenPose = midDrillPose , 
        zApproach=0.20, 
        zClose=0.040, 
        Tpliant = 0.3 ,  
        wdthNarrow = 0.038 , maxIter=5, drillPress=_DRILL_CONTACT_PRESS , 
        zFree= _LONG_DRILL_SAFE_Z , # <-- VERY IMPORTANT - AVOID BREAKING DRILL
        driverLen = 0.160 ,
        ctrl = robot 
    )
)

run_BT_until_done( testTree , N = 90000 , 
                   tickPause = 0.25 , 
                   breakOnFailure = 1 , breakOnSuccess = 1 , 
                   Nverb = 100 )

### RUN BT Sequence ###

def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 14.328123807907104 seconds!
Halt_Drill: Ended with drill ATTACHED!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 14.417784690856934 seconds!
Halt_Drill: Ended with drill ATTACHED!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 14.446201086044312 seconds!
Halt_Drill: Ended with drill ATTACHED!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 12.132194995880127 seconds!
Halt_Drill: Ended with drill ATTACHED!
def condition_false():
    """ Always return False """
    return False

Maintain_Z_Pressure: self.ctrl.maintain_z_pressure ran for 14.378255128860474 seconds!
Halt_Drill: Ended w